# Social graph and Interactions

# Assignment 2

**Group 30:**

Aleksandar Lukic, s194066

Clara Mejlhede Lorenzen, s180350

Julia Katarzyna Zalewska, s244491

## Python libraries used in this assignment

In [7]:
import json
import math
import matplotlib as mpl
from matplotlib import colormaps
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import re
import urllib.request
import pandas as pd
#%pip install louvain
#import community 

## Part 1: Genres and communities and plotting

#### <font color='gray'>  Write about genres and modularity. </font>

Genres is a cluster of communities in a network. Modularity is a measure of how well a network can be divided into communities. A network with high modularity has dense connections between nodes in the same community and sparse connections between nodes in different communities.


<font color='gray'>  Work from the undirected version of the network, keeping only the nodes for which you have genre information in the file above. </font>

In [8]:
# Load the data
url = 'https://raw.githubusercontent.com/SocialComplexityLab/socialgraphs2024/refs/heads/main/files/artists_genres_dictionary.txt'
data = pd.read_json(url, orient='index')

# Convert to dictionary
genres = data.to_dict()[0]  

# Initialize new undirected graph
G = nx.Graph()

# Add nodes with genre information
for artist, genre_list in genres.items():
    G.add_node(artist, genres=genre_list)

# Add edges between artists who share genres
for artist1, genres1 in genres.items():
    for artist2, genres2 in genres.items():
        if artist1 != artist2:
            common_genres = set(genres1).intersection(genres2)
            if common_genres:  
                G.add_edge(artist1, artist2, weight=len(common_genres))  

# Subgraph containing only artists in dataset
artists = list(genres.keys())
G = G.subgraph(artists)

# Print graph information
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

Graph has 1833 nodes and 1657291 edges.


<font color='gray'> Now create your own partition of the network into genres. </font>

In [9]:
# Create a partition of the network into genres
partition = {}
for artist, genre_list in genres.items():
    partition[artist] = genre_list[0]

# Print the partition
print(partition)

{'A Thousand Horses': 'c', 'A.P. Carter': 'c', 'Aaron Barker': 'c', 'Aaron Benward': 'c', 'Aaron Lewis (musician)': 'a', 'Aaron Lines': 'c', 'Aaron Neville': 's', 'Aaron Pritchett': 'c', 'Aaron Watson': 'c', 'Ace in the Hole Band': 'a', 'Adam Brand (musician)': 'c', 'Adam Gregory': 'c', 'Adam Harvey': 'c', 'Adrienne Young': 'b', 'Al Anderson (NRBQ)': 'r', 'Al Caiola': 'j', 'Al Dexter': 'c', 'Al Downing (musician)': 'r', 'Al Hurricane Jr.': 'n', 'Al Hurricane': 'n', 'Alabama (band)': 'c', 'Alan Jackson': 'n', 'Alecia Elliott': 'c', 'Alecia Nugent': 'b', 'Alee (singer)': 'c', 'Aleyce Simmonds': 'c', 'Alisan Porter': 'p', 'Alison Krauss': 'b', 'Alli Walker': 'c', 'Allison Moorer': 'a', 'Allison Russell': 'a', 'Allman Brothers Band': 's', 'Alyssa Micaela': 'c', 'Amanda Shires': 'a', 'Amanda Stott': 'c', 'Amanda Wilkinson': 'c', 'Amazing Rhythm Aces': 'c', 'Amber Dotson': 'c', 'Amber Hayes': 'c', 'Amber Lawrence': 'a', 'American Young': 'c', 'Amie Comeaux': 'c', 'Amy Dalley': 'c', 'Amy Gran

<font color='gray'> Spoiling your findings a little bit, an issue is the following: Almost all artists have the genre country as their first genre in the list, so by definition the network won't have very high modularity. What happens to the modularity if you grab the first genre that's not country as the genre-assignment for all nodes that have more than one genre? </font>

In [10]:
# Create a partition of the network into genres
partition = {}
for artist, genre_list in genres.items():
    if len(genre_list) > 1:
        partition[artist] = genre_list[1]
    else:
        partition[artist] = genre_list[0]

# Print the partition
print(partition)

{'A Thousand Horses': 'o', 'A.P. Carter': 'o', 'Aaron Barker': 'o', 'Aaron Benward': 'o', 'Aaron Lewis (musician)': 'l', 'Aaron Lines': 'o', 'Aaron Neville': 'o', 'Aaron Pritchett': 'o', 'Aaron Watson': 'o', 'Ace in the Hole Band': 'l', 'Adam Brand (musician)': 'o', 'Adam Gregory': 'o', 'Adam Harvey': 'o', 'Adrienne Young': 'l', 'Al Anderson (NRBQ)': 'o', 'Al Caiola': 'a', 'Al Dexter': 'o', 'Al Downing (musician)': 'o', 'Al Hurricane Jr.': 'e', 'Al Hurricane': 'e', 'Alabama (band)': 'o', 'Alan Jackson': 'e', 'Alecia Elliott': 'o', 'Alecia Nugent': 'l', 'Alee (singer)': 'o', 'Aleyce Simmonds': 'o', 'Alisan Porter': 'o', 'Alison Krauss': 'l', 'Alli Walker': 'o', 'Allison Moorer': 'm', 'Allison Russell': 'm', 'Allman Brothers Band': 'o', 'Alyssa Micaela': 'o', 'Amanda Shires': 'm', 'Amanda Stott': 'c', 'Amanda Wilkinson': 'o', 'Amazing Rhythm Aces': 'o', 'Amber Dotson': 'o', 'Amber Hayes': 'o', 'Amber Lawrence': 'u', 'American Young': 'o', 'Amie Comeaux': 'o', 'Amy Dalley': 'o', 'Amy Gran

#### <font color='gray'> Detect the communities, discuss the value of modularity in comparison to the genres.</font>

<font color='gray'> Use the Python Louvain-algorithm implementation to find communities each network. Report the value of modularity found by the algorithm. Is it higher or lower than what you found above for the genres as communities? What does this comparison reveal about them?</font>

In [12]:
# Find communities in the network
communities = community.best_partition(G)

# Print the modularity of the partition
modularity = community.modularity(communities, G)
print(f"Modularity: {modularity}")

# Create a new graph with the communities as nodes
H = nx.Graph()
for artist, community_id in communities.items():
    H.add_node(artist, community=community_id)

# Add edges between communities
for artist1, artist2 in G.edges():
    community1 = communities[artist1]
    community2 = communities[artist2]
    if community1 != community2:
        H.add_edge(community1, community2)

# Print graph information
print(f"Graph has {H.number_of_nodes()} nodes and {H.number_of_edges()} edges.")

# Create a partition of the network into communities
partition = {}
for artist, community_id in communities.items():
    partition[artist] = community_id


NameError: name 'community' is not defined

It should come with this outcome: 

Modularity: 0.03843062171909256

Graph has 1835 nodes and 1 edges.

<font color='gray'>What is the modularity of this partitioning of the network? Is it more community-like than the genres?</font>

Text

<font color='gray'>Visualize the network, using the Force Atlas algorithm</font>

In [ ]:
pos = nx.spring_layout(G, k=0.1)

# Draw the nodes and edges
nx.draw_networkx_nodes(G, pos, node_size=10, node_color='blue', alpha=0.5)
nx.draw_networkx_edges(G, pos, alpha=0.1)

# Draw the labels
nx.draw_networkx_labels(G, pos, font_size=8)

# Show the plot
plt.axis('off')
plt.show()

#### <font color='gray'> Calculate the matrix $D$ and discuss your findings.</font>

<font color='gray'> Compare the communities found by your algorithm with the music genres by creating a matrix </font> $D$ <font color='gray'> with dimension  </font> ($G\times C$), <font color='gray'> where </font> $G$ <font color='gray'> is the 7 most common genres and </font> $C$ <font color='gray'> is the 7 most common communities. We set </font> $D(i,j)$ <font color='gray'> entry to be the number of nodes that genre </font> $i$ <font color='gray'> has in common with community </font> $j$. <font color='gray'> The matrix </font> $D$ <font color='gray'> is what we call a confusion matrix. </font>

<font color='gray'>  Use the confusion matrix to explain how well the communities you've detected correspond to the genres. </font>

#### <font color='gray'> Plot the communities and comment on your results.</font>

## Part 2: TF-IDF to understand genres and communities

#### <font color='gray'> Explain the concept of TF-IDF in your own words and how it can help you understand the genres and communities. </font>

#### <font color='gray'> Calculate and visualize TF-IDF for the genres and communities.</font>

#### <font color='gray'> Use the matrix D (Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.</font>

## Part 3: Sentiment of the artists and communities

#### <font color='gray'> Calculate the sentiment of the Artists pages (OK to work with the sub-network of artists-with-genre) and describe your findings using stats and visualization, inspired by the first exercise of week 8.</font>

#### <font color='gray'> Discuss the sentiment of the largest communities. Do the findings using TF-IDF during Lecture 7 help you understand your results?</font>